In [6]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
!pip install jedi
!pip install google-api-python-client
!pip install google-auth-httplib2
!pip install google-auth-oauthlib

from IPython.display import clear_output
clear_output()

In [19]:
import pickle, os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaFileUpload
from mimetypes import guess_type
import libtorrent as lt
import time
from IPython.display import display
import ipywidgets as widgets
import os



class GoogleDriveTorrent:

    def __init__(self):
        self.drive_service = self.service()

    def slash(self, i:str):
        return i.replace('\\','/')

    def service(self):
        creds = None
        try:
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        except:
            pass
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file('credentials.json', 
                ['https://www.googleapis.com/auth/drive.metadata.readonly','https://www.googleapis.com/auth/drive.file'])
                creds = flow.run_local_server(port=0)
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)
        return build('drive', 'v3', credentials=creds)

    def create_folder(self, directory_name, parent_id):
            file_metadata = {"name": directory_name, "mimeType": "application/vnd.google-apps.folder"}
            file_metadata["parents"] = [parent_id]
            file = self.drive_service.files().create(supportsTeamDrives=True, body=file_metadata).execute()
            return file

    def get_mime_type(self, file_path):
            mime_type = guess_type(file_path)
            return mime_type if mime_type[0] else "text/plain"


    def upload_file(self, filepath, filename, parent_id):
        mime_type = self.get_mime_type(filepath)
        print(mime_type)
        file_metadata = {'name': filename,"parents": [parent_id]}
        media = MediaFileUpload(filepath,mimetype=mime_type)
        file = self.drive_service.files().create(supportsTeamDrives=True,
        body=file_metadata,media_body=media, fields='id').execute()
        return file

    def upload_dir(self, input_directory, parent_id, first_parent=False):
        if first_parent:
          parent_id = self.create_folder(first_parent,parent_id)['id']
        list_dirs = os.listdir(input_directory)
        if len(list_dirs) == 0:
            return parent_id
        new_id = None
        for item in list_dirs:
            current_file_name = os.path.join(input_directory, item)
            if os.path.isdir(current_file_name):
                current_dir = self.create_folder(item, parent_id)
                new_id = self.upload_dir(current_file_name, current_dir.get('id'))
            else:
                file_name = str(self.slash(current_file_name)).split('/')[-1]
                self.upload_file(current_file_name, file_name, parent_id)
                new_id = parent_id
        return new_id

    def torrent_downloader(self, tf: str, files: list = []):
        ses = lt.session()
        ses.listen_on(6881, 6891)
        if(tf.startswith("magnet")):
            params = {
                "save_path": "."
            }

            handle = lt.add_magnet_uri(ses, tf, params)
        else: 
            os.system(f"wget -q {tf}")
            source = tf.split("/")[-1]
            params = {
                "save_path": ".",
                "ti": lt.torrent_info(source),}
            handle = ses.add_torrent(params)
            os.system(f"rm \"{source}\"")
        info = handle.get_torrent_info()
        all_file = [dl_file.path.split("/") for index, dl_file in enumerate(list(info.files()))]
        pr = handle.get_file_priorities()
        if files:
            for index, dl_file in enumerate(list(info.files())):
                file_name = dl_file.path.split("/")[-1]
                if file_name in files:
                    files.remove(file_name)
                else:
                    pr[index] = 0
            for rf in files:
                print(f"{rf} NOT FOUND!")
        handle.prioritize_files(pr)
        
        state_str = [
            "Queued",
            "Checking",
            "Downloading MetaData",
            "Downloading",
            "Finished",
            "Seeding",
            "Allocating",
            "Checking FastResume",
        ]

        layout = widgets.Layout(width="auto")
        style = {"description_width": "initial"}
        bar = widgets.FloatSlider(
            step=0.01, disabled=True, layout=layout, style=style
        )
        display(bar)
        
        while (handle.status().state != lt.torrent_status.seeding):
            s = handle.status()
            if s.state == lt.torrent_status.downloading:
                bar.description = " | ".join(
                    [   
                        state_str[s.state],
                        f"{s.total_wanted_done/1024/1024:.1f}MB/{s.total_wanted/1024/1024:.1f}MB",
                        f"Speed: {s.download_rate/1024:.2f} KB/s",
                    ]
                )
            else:
                bar.description = " ".join(
                    [   
                        state_str[s.state]
                    ]
                )
            bar.value = s.progress * 100
            if s.progress == 1:
                bar.description = " ".join(
                    [   
                        state_str[lt.torrent_status.finished]
                    ]
                )
                break
        ses.remove_torrent(handle)
        return all_file


In [ ]:
drive_id = 'YOUR_DRIVE_ID'
torent = 'https://nyaa.si/download/1491641.torrent'


gt = GoogleDriveTorrent()
i = gt.torrent_downloader(tf=torent, files=[])
if len(i) == 1: gt.upload_file(f'./{i[0][0]}',i[0][0],drive_id)
else: gt.upload_dir(i[0][0], drive_id, i[0][0]) 